# import libraries 

In [286]:
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import random

import matplotlib.pyplot as plt


2.2.0-rc3


In [0]:
# # colab access to google drive 

# from google.colab import drive
# drive.mount('/content/gdrive')

In [0]:
# # Visit www.kaggle.com ⇨ login ⇨ My Account ⇨ Create New API Token
# from google.colab import files
# files.upload() 

# from google.colab import files
# files.upload() #this will prompt you to update the json

# !pip install -q kaggle
# !mkdir -p ~/.kaggle
# !cp kaggle.json ~/.kaggle/
# !ls ~/.kaggle
# !chmod 600 /root/.kaggle/kaggle.json  # set permission

In [0]:
# # syntax: !kaggle competitions download -c ‘name_of_competition’ -p “target_colab_dir”
# !kaggle competitions download -c histopathologic-cancer-detection -p /content/gdrive/My\ Drive/kaggle/cancer

# # create dirs
# os.chdir('gdrive/My Drive/kaggle/cancer')  #change dir
# !mkdir train  #create a directory named train/
# !mkdir test  #create a directory named test/

# # !unzip -q file[.zip] -d [exdir]
# !unzip -q train.zip -d train/  #unzip data in train/
# !unzip -q test.zip -d test/  #unzip data in test/
# !unzip sample_submission.csv.zip
# !unzip train_labels.csv.zip

# define helper functions 

In [0]:
def plot_graphs(history, metric):
  val = 'val_'+metric
  plt.plot(history.history[metric], label=metric)
  plt.plot(history.history[val], label=val)
  plt.xlabel('time')
  plt.ylabel(metric)
  plt.grid(True)
  
  # Place a legend to the right of this smaller subplot.
  plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)



# get dataset

In [0]:
%%capture
!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/horse-or-human.zip \
    -O /tmp/horse-or-human.zip

!wget --no-check-certificate \
    https://storage.googleapis.com/laurencemoroney-blog.appspot.com/validation-horse-or-human.zip \
    -O /tmp/validation-horse-or-human.zip

In [0]:
# # unzip dataset= !unzip file_path -d target_path

# !unzip '/tmp/horse-or-human.zip' -d '/tmp/horse-or-human'
# !unzip '/tmp/validation-horse-or-human.zip' -d '/tmp/validation-horse-or-human'

# define training & validation dir

In [0]:
base_dir = os.path.join('/tmp')
train_dir = os.path.join(base_dir, 'horse-or-human')
validation_dir = os.path.join(base_dir, 'validation-horse-or-human')

# shuffle datasets to reduce sequence bias

In [294]:
train_horses_dir = os.path.join(train_dir, 'horses')
train_humans_dir = os.path.join(train_dir, 'humans')
valid_horses_dir = os.path.join(validation_dir, 'horses')
valid_humans_dir = os.path.join(validation_dir, 'humans')

train_horses_fnames = os.listdir(train_horses_dir)
train_humans_fnames = os.listdir(train_humans_dir)
valid_horses_fnames = os.listdir(valid_horses_dir)
valid_humans_fnames = os.listdir(valid_humans_dir)

random.shuffle(train_horses_fnames)
random.shuffle(train_humans_fnames)
random.shuffle(valid_horses_fnames)
random.shuffle(valid_humans_fnames)
print('horses: {}'.format(train_horses_fnames))
print('humans: {}'.format(train_humans_fnames))
print('valid_horses: {}'.format(valid_horses_fnames))
print('valid_humans: {}'.format(valid_humans_fnames))

horses: ['horse03-8.png', 'horse32-4.png', 'horse09-5.png', 'horse29-6.png', 'horse01-3.png', 'horse46-0.png', 'horse32-1.png', 'horse23-7.png', 'horse17-8.png', 'horse28-3.png', 'horse46-8.png', 'horse38-0.png', 'horse30-8.png', 'horse50-0.png', 'horse47-8.png', 'horse38-7.png', 'horse14-2.png', 'horse45-4.png', 'horse48-1.png', 'horse46-1.png', 'horse01-9.png', 'horse25-4.png', 'horse18-6.png', 'horse15-8.png', 'horse27-6.png', 'horse30-5.png', 'horse40-9.png', 'horse14-0.png', 'horse26-9.png', 'horse01-1.png', 'horse01-6.png', 'horse06-5.png', 'horse39-3.png', 'horse05-5.png', 'horse16-8.png', 'horse21-8.png', 'horse04-8.png', 'horse20-3.png', 'horse42-3.png', 'horse17-6.png', 'horse49-2.png', 'horse35-8.png', 'horse17-1.png', 'horse16-7.png', 'horse26-6.png', 'horse29-0.png', 'horse13-2.png', 'horse44-8.png', 'horse25-0.png', 'horse44-2.png', 'horse49-9.png', 'horse26-0.png', 'horse24-6.png', 'horse07-8.png', 'horse37-5.png', 'horse13-9.png', 'horse20-1.png', 'horse34-7.png', 'hors

# preprocessing 

image augmentation is where you adjust the training set to potentially increase the number of diverse subjects it encompasses.
- image augmentation simply amends your images while training using transformations like rotation to simulate different subjects.

image augmentation potentially resolves overfitting by providing a greater span of different images to train on. 

---

**overfitting** 

when a dataset is small, we have few examples, potentially causing overfitting: 

`high classificaiton accuracy on a limited dataset, but significantly inferior classification accuracy on a general dataset`

In [295]:

# instantiate ImageDataGenerator with params
train_datagen = ImageDataGenerator(
    rescale=1./255, # normalize
    rotation_range=40, # randomly rotate (0-180 degrees)
    width_shift_range=0.2, # move img horizontally within frame
    height_shift_range=0.2, # move img vertically within frame
    shear_range=0.2, # transform img to cut out sections (0.2=20%)
    zoom_range=0.2, # magnify % of img
    horizontal_flip=True, # randomly make multiple imgs structurally similar
    fill_mode='nearest' # fill in lost pixels via transformation. 
)

validation_datagen = ImageDataGenerator(
    rescale=1./255, # normalize
    rotation_range=40, # randomly rotate (0-180 degrees)
    width_shift_range=0.2, # move img horizontally within frame
    height_shift_range=0.2, # move img vertically within frame
    shear_range=0.2, # transform img to cut out sections (0.2=20%)
    zoom_range=0.2, # magnify % of img
    horizontal_flip=True, # randomly make multiple imgs structurally similar
    fill_mode='nearest' # fill in lost pixels via transformation. 
)

# flow the training imgs to generator in batches of 20 using train_datagen
train_generator = train_datagen.flow_from_directory(
    train_dir, # source of training imgs
    target_size=(300,300), # resize all imgs to 300x300
    batch_size=20, 
    class_mode='binary'
)

# flow the validation imgs to generator in batches of 20 using validation_datagen
validation_generator = validation_datagen.flow_from_directory(
    validation_dir, # source of validation imgs
    target_size=(300, 300), # resize all imgs to 300x300
    # always batch size shoudld always cover as much as dataset as possible with steps 
    batch_size=42,
    class_mode='binary'
)


Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


# get pre-trained model

In [296]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

--2020-04-26 17:59:24--  https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 87910968 (84M) [application/x-hdf]
Saving to: ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’

/tmp/inception_v3_w 100%[===================>]  83.84M   228MB/s    in 0.4s    

2020-04-26 17:59:24 (228 MB/s) - ‘/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5’ saved [87910968/87910968]



In [0]:
# reset memory values to prevent side-effects
tf.keras.backend.clear_session()

# transfer learning 

transfer learning allows you to retrain the final layer of an existing model.

being able to retrain the final layer means that you can maintain the knowledge that the model had learned during its original training and apply it to your smaller dataset to achieve a specific task. 

`transfer learning results in highly accurate classifications without the need for extensive training time and computational power.`


In [0]:
from tensorflow.keras import layers # inspect layers

# InceptionV3 by Google was trained using a dataset of 1,000 classes
from tensorflow.keras.applications.inception_v3 import InceptionV3 

# pre-trained model weights
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# define pre-trained model 
pre_trained_model = InceptionV3(
    # img pixel dimensions = 300 height x 300 width x 3 color_channels
    input_shape=(300, 300, 3),
    include_top=False, # top layer is used for retraining for a specific task
    weights=None
)

# load pre-trained weights on to InceptionV3 model
pre_trained_model.load_weights(local_weights_file)

# pre-trained model freezing  

for transfer learning, freeze the weights (trainable=False) of an existing model and add specialized (trainable=True) top layer 

In [0]:
# loop through model layers and freeze
for layer in pre_trained_model.layers:
  layer.trainable = False

# model.summary()

# inspect pre-trained model output layer

In [300]:
last_layer = pre_trained_model.get_layer('mixed7') # can get layer name from model summary
print('top layer output_shape: {}'.format(last_layer.output_shape))
last_output = last_layer.output


top layer output_shape: (None, 17, 17, 768)


# define model with specialized layers 

**transfer learning**

`add specialized layers that will be on top of the frozen layers of the pre-trained model`

---

**dropout layer**

`dropout layer is useful in preventing overfitting by deactivating a random number of neurons in a layer`

adding a dropout layer deters neighbor neurons from having similar weights, thus preserving final training
- a neuron taking in multiple inputs with varying weights cannot rely on any one feature, motivating it to spread out the weights. 

---

**regularization**

in terms of deep learning, `regularization is a technique uses to prevent overfitting` -- training accuracy increasing, but testing accuracy decreases.

- L1 regularization (lasso regression) removes seamingly unimportant features to prevent overfitting, but potentially causing underfitting.

- L2 regularization (ridge regression) gives each feature different weights of importance & influence.


In [0]:
# formatting layer(x) is an alt method to building a model

x = tf.keras.layers.Flatten()(last_output) # flatten input_vector (InceptionV3 pre-trained model output)
# to prevent apparent overfitting
  # started at 1024 weights, reduced to 512
  # applied L2 regularization hyperparameter to focus on more successful weights
  # dropout= randomly deactivate 30% of neurons in this layer
  # increased training time from 20 epochs => 50
  # decreased learning rate from 1e-4 => 1e-5
x = tf.keras.layers.Dense(units=512, activation='relu', 
                          activity_regularizer=tf.keras.regularizers.l2(0.01))(x)
x = tf.keras.layers.Dropout(0.25)(x) 

# output layer/ transfer learning top layer

### binomial
x = tf.keras.layers.Dense(units=1, activation='sigmoid')(x)

### multinomial
# x = tf.keras.layers.Dense(units=2, activation='softmax')(x)


# add specialized layers on top of pre_trained_model layers
model = tf.keras.Model(pre_trained_model.input, x)

# model.summary()

# compile the model

build the model by compiling it with a loss, optimizer, and objective metric
- the optimizer uses the loss to adjust the weights to optimize prediction accuracy
- the loss=prediction accuracy
- metrics= target performance indicators

In [0]:
# since its a binary classifier, use binary crossentropy loss
# lr = learning rate/alpha 0.00001 = good
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), metrics=['accuracy'])
# model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.RMSprop(lr=0.00001), metrics=['accuracy'])

# changing optimizer to Adam or SGD yields terrible resutls 
# model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(lr=0.0001), metrics=['accuracy'])
# model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.0001), metrics=['accuracy'])

# define callbacks 

learningRateScheduler:

`the learning rate scheduler will adjust the learning rate for each epoch`


In [0]:
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-8 * 10**(epoch/20))

# enable early_stopping
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if logs.get('val_accuracy') >= 0.98 and logs.get('accuracy') >= 0.98:
      self.model.stop_training = True

callbacks = myCallback()

In [304]:
history = model.fit(
    train_generator, # preprocessed dataset
    steps_per_epoch=8, # 1027 train imgs = batch_size(32) * steps(8)
    epochs=50,
    validation_data=validation_generator,
    # if overfitting, consider adjusting batch_size of validation_dataset
      # smaller batch_sizes are better and finding global minimum in gradient descent
    validation_steps=6,# 255 test imgs = batch_size(32) * steps(8) (closest to 256)
    callbacks=[callbacks],
    verbose=1
)

Epoch 1/50
8/8 [==============================] - 10s 1s/step - loss: 2.9059 - accuracy: 0.6375 - val_loss: 0.6266 - val_accuracy: 0.7778
Epoch 2/50
8/8 [==============================] - 9s 1s/step - loss: 0.5739 - accuracy: 0.9000 - val_loss: 0.6708 - val_accuracy: 0.6746
Epoch 3/50
8/8 [==============================] - 9s 1s/step - loss: 0.5094 - accuracy: 0.9750 - val_loss: 0.5674 - val_accuracy: 0.9405
Epoch 4/50
8/8 [==============================] - 9s 1s/step - loss: 0.4958 - accuracy: 0.9750 - val_loss: 0.5483 - val_accuracy: 0.9127
Epoch 5/50
8/8 [==============================] - 9s 1s/step - loss: 0.5185 - accuracy: 0.9750 - val_loss: 0.5602 - val_accuracy: 0.9444
Epoch 6/50
8/8 [==============================] - 9s 1s/step - loss: 0.4848 - accuracy: 1.0000 - val_loss: 0.7481 - val_accuracy: 0.6508
Epoch 7/50
8/8 [==============================] - 9s 1s/step - loss: 0.5592 - accuracy: 0.9252 - val_loss: 0.6200 - val_accuracy: 0.9167
Epoch 8/50
8/8 [========================

# visualize performance 

In [0]:
plt.figure(figsize=(10,6))
plot_graphs(history, 'accuracy')

print("mae: {}".format(tf.keras.metrics.mean_absolute_error(history.history['val_accuracy'], history.history['accuracy'])))

# model save 

In [0]:
model.save('cnn-imgDataGen-transfer_learning-horses_v_humans.h5')

# clean up

In [0]:
import os, signal

os.kill(os.getpid(), signal.SIGKILL)